# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 4. Klasifikasi Naive Bayes


Tuliskan Nama, NIM, dan kelas Anda:

Nama  : Dani Adrian

NIM   : 225150201111009

Kelas : PPM E

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [131]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-03-31 20:18:47--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.6’

iris.csv.6          100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-03-31 20:18:48 (53.9 MB/s) - ‘iris.csv.6’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [132]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [133]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [134]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [135]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [136]:
label_uji = data_uji.pop('variety')

In [137]:
data_uji.head()

,sepal.length,sepal.width,petal.length,petal.width
49,5.0,3.3,1.4,0.2
78,6.0,2.9,4.5,1.5
30,4.8,3.1,1.6,0.2
93,5.0,2.3,3.3,1.0
146,6.3,2.5,5.0,1.9


## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data.

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [138]:
from collections import Counter

def hitung_prior(list_kelas):
    n_data = len(list_kelas)
    prior = Counter(list_kelas)
    for key in prior.keys():
        prior[key] = prior[key] / n_data
    return prior

Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [139]:
prior = hitung_prior(data_latih['variety'])
print(prior)

Counter({'Virginica': 0.35, 'Setosa': 0.3333333333333333, 'Versicolor': 0.31666666666666665})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [140]:
def hitung_rata2_std_kelas(input_data):
    list_columns = input_data.columns[:-1]
    class_column_name = input_data.columns[-1]
    list_class = set(input_data[class_column_name])
    rata2 = {}
    std = {}
    for column in list_columns:
        for a_class in list_class:
            rata2[(a_class, column)] = input_data.loc[input_data[class_column_name] == a_class][column].mean()
            std[(a_class, column)] = input_data.loc[input_data[class_column_name] == a_class][column].std()
    return rata2, std

Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [141]:
import math

def hitung_likelihood_gaussian(data, rata2, std):
    hasil = (1 / (math.sqrt(2 * math.pi * (std ** 2)))) * math.exp((-1 * ((data - rata2) ** 2)) / (2 * (std ** 2)))
    return hasil

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-msaing fitur dan kelas.

In [142]:
def training_naive_bayes_gaussian(data_latih):
    class_column_name = data_latih.columns[-1]
    prior = hitung_prior(data_latih[class_column_name])
    (rata2, std) = hitung_rata2_std_kelas(data_latih)
    list_class = set(data_latih[class_column_name])
    list_columns = data_latih.columns[:-1]

    model = {}
    model['prior'] = prior
    model['rata2'] = rata2
    model['std'] = std
    model['list_class'] = list_class
    model['list_columns'] = list_columns

    return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [143]:
model = training_naive_bayes_gaussian(data_latih)
print(model)

{'prior': Counter({'Virginica': 0.35, 'Setosa': 0.3333333333333333, 'Versicolor': 0.31666666666666665}), 'rata2': {('Setosa', 'sepal.length'): 4.9625, ('Versicolor', 'sepal.length'): 5.978947368421053, ('Virginica', 'sepal.length'): 6.642857142857143, ('Setosa', 'sepal.width'): 3.3825000000000003, ('Versicolor', 'sepal.width'): 2.7473684210526312, ('Virginica', 'sepal.width'): 2.969047619047619, ('Setosa', 'petal.length'): 1.4475000000000002, ('Versicolor', 'petal.length'): 4.310526315789474, ('Virginica', 'petal.length'): 5.585714285714285, ('Setosa', 'petal.width'): 0.24, ('Versicolor', 'petal.width'): 1.3289473684210522, ('Virginica', 'petal.width'): 2.0285714285714285}, 'std': {('Setosa', 'sepal.length'): 0.3432032364918221, ('Versicolor', 'sepal.length'): 0.481640156546662, ('Virginica', 'sepal.length'): 0.5967974927785468, ('Setosa', 'sepal.width'): 0.35869242591704725, ('Versicolor', 'sepal.width'): 0.3064503329144635, ('Virginica', 'sepal.width'): 0.30161200664955035, ('Setosa'

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [144]:
def testing_naive_bayes_gaussian(model, data_uji):
    prior = model['prior']
    rata2 = model['rata2']
    std = model['std']
    list_class = model['list_class']
    list_columns = model['list_columns']
    posterior = dict.fromkeys(list_class, 1)

    for a_class in list_class:
        for column in list_columns:
            posterior[a_class] *= hitung_likelihood_gaussian(data_uji[column], rata2[(a_class, column)], std[(a_class, column)])
        posterior[a_class] *= prior[a_class]

    kelas_uji = max(posterior, key=posterior.get)
    return kelas_uji

Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [145]:
indeks_uji = 0
prediksi = testing_naive_bayes_gaussian(model, data_uji.iloc[indeks_uji])
print(prediksi)
print(label_uji.iloc[indeks_uji])

Setosa
Setosa


Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [146]:
prediksi_total = []
for indeks in range(data_uji.shape[0]):
    prediksi_total.append(testing_naive_bayes_gaussian(model, data_uji.iloc[indeks]))

total_benar = sum(prediksi_total[i] == label_uji.iloc[i] for i in range(len(prediksi_total)))
print("Total prediksi benar: ", total_benar)

Total prediksi benar:  29


## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

In [147]:
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    print('File berhasil diupload:', filename)

Saving car_sample.csv to car_sample (1).csv
File berhasil diupload: car_sample (1).csv


In [148]:
import pandas as pd
data_tugas = pd.read_csv('car_sample.csv')
data_tugas.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [149]:
tugas_latih,tugas_uji = train_test_split(data_tugas,test_size=0.3)
print(tugas_latih.shape[0])
print(tugas_uji.shape[0])

1209
519


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [150]:
tugas_label_uji = tugas_uji.pop('class')
tugas_uji.head()

,buying,maint,lug_boot,safety
1725,low,low,big,low
842,high,low,med,high
1708,low,low,big,med
1041,med,high,big,low
65,vhigh,vhigh,small,high


Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [151]:
prior_tugas = hitung_prior(tugas_latih['class'])
print(prior_tugas)

Counter({'unacc': 0.695616211745244, 'acc': 0.22084367245657568, 'vgood': 0.04218362282878412, 'good': 0.0413564929693962})


Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [152]:
def likelihood_count(data_latih, a_class, column, values):
  column_values = data_latih[column] == values
  class_values = data_latih['class'] == a_class
  hasil = len(data_latih[(column_values)&(class_values)])+1/(data_latih[column].count())
  return hasil

In [153]:
def hitung_likelihood_multinomial(data_latih):
  list_columns = data_latih.columns[:-1]
  class_column_name = data_latih.columns[-1]
  list_class = set(data_latih[class_column_name])
  likelihood = {}
  for column in list_columns:
    for val in data_latih[column].unique():
      for a_class in list_class:
        likelihood[(column, val, a_class)] = (likelihood_count(data_latih, a_class, column, val))
  return likelihood

Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [154]:
likelihood = hitung_likelihood_multinomial(tugas_latih)
print(likelihood)

{('buying', 'vhigh', 'acc'): 43.00082712985939, ('buying', 'vhigh', 'vgood'): 0.0008271298593879239, ('buying', 'vhigh', 'good'): 0.0008271298593879239, ('buying', 'vhigh', 'unacc'): 256.0008271298594, ('buying', 'low', 'acc'): 58.00082712985939, ('buying', 'low', 'vgood'): 33.00082712985939, ('buying', 'low', 'good'): 34.00082712985939, ('buying', 'low', 'unacc'): 181.00082712985937, ('buying', 'med', 'acc'): 81.00082712985939, ('buying', 'med', 'vgood'): 18.00082712985939, ('buying', 'med', 'good'): 16.00082712985939, ('buying', 'med', 'unacc'): 183.00082712985937, ('buying', 'high', 'acc'): 85.00082712985939, ('buying', 'high', 'vgood'): 0.0008271298593879239, ('buying', 'high', 'good'): 0.0008271298593879239, ('buying', 'high', 'unacc'): 221.00082712985937, ('maint', 'low', 'acc'): 65.00082712985939, ('maint', 'low', 'vgood'): 18.00082712985939, ('maint', 'low', 'good'): 33.00082712985939, ('maint', 'low', 'unacc'): 185.00082712985937, ('maint', 'vhigh', 'acc'): 48.00082712985939, 

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [155]:
def training_naive_bayes_multinomial(tugas_latih):
  #tulis kode Anda disini
  class_column_name = tugas_latih.columns[-1]
  list_class = set(tugas_latih[class_column_name])
  list_columns = tugas_latih.columns[:-1]
  model = {}
  model['prior'] = prior_tugas
  model['likelihood'] = likelihood
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [156]:
model_tugas = training_naive_bayes_multinomial(tugas_latih)

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [157]:
def testing_naive_bayes_multinomial(model,tugas_uji):
  #tulis kode Anda disini
  prior = model['prior']
  likelihood = model['likelihood']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class, 1)
  for a_class in list_class:
    for column in list_columns:
      posterior[a_class] = posterior[a_class] * likelihood[column, tugas_uji[column], a_class]
      posterior[a_class] = posterior[a_class] * prior[a_class]
      kelas_uji = max(posterior, key = posterior.get)
  return kelas_uji

Lakukan pengujian menggunakan sebuah data uji

In [158]:
idx = 10
uji = tugas_uji.iloc[idx]
kelas_uji = testing_naive_bayes_multinomial(model_tugas,uji)
print(kelas_uji)
print(tugas_label_uji.iloc[idx])

unacc
unacc


Lakukan pengujian terhadap semua data uji.

In [159]:
prediksi_total =[]
for indeks in range(tugas_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_multinomial(model_tugas,tugas_uji.iloc[indeks]))
print("Total prediksi benar: ",sum(prediksi_total==tugas_label_uji))

Total prediksi benar:  369


In [160]:
len(tugas_label_uji)

519